In [ ]:
import scanpy as sc
from matplotlib.patches import Rectangle
import matplotlib.pyplot as plt
import numpy as np
import warnings
from scipy.spatial import KDTree


warnings.filterwarnings("ignore")

In [ ]:
adata = sc.read_h5ad("../data/adata/timecourse.h5ad")

In [ ]:
batches = {
    # "day6_SI": {"x": 6200, "y": 6200},
    # "day6_SI_r2": {"x": 5800, "y": 5500},
    # "day8_SI_Ctrl": {"x": 2400, "y": 2400},
    "day8_SI_r2": {"x": 3200, "y": 1500},
    # "day30_SI": {"x": 6400, "y": 2400},
    # "day30_SI_r2": {"x": 6200, "y": 6200},
    # "day90_SI": {"x": 2400, "y": 2400},
    # "day90_SI_r2": {"x": 1200, "y": 6200},
}

# subset anndata
adata = adata[adata.obs.batch.isin(batches.keys())]
adata

In [ ]:
colors = {
    "Epithelial_Secretory": "#AA9228",
    "Epithelial_Absorptive": "#E3C300",
    "Monocyte": "#C37698",
    "T-Cell": "#008E74",
    "MAIT": "#63ABB9",
    "Myeloid": "#EF9684",
    "ILC": "#A0C6D3",
    "B-Cell": "#E2CEAB",
    "DC": "#FE757D",
    "Fibroblast": "#E17300",
    "Endothelial": "#E30133",
    "NK": "#4A7B89",
    "Epithelial_Progenitor": "#F7BC00",
    "Neuron": "#2A2446",
    "Erythroid": "#A5021D",
    "Eosinophil": "#782c4e",
}

Defining the area to zoom in on

In [ ]:
xmin, xmax = 555 * 50 * 0.2125, 600 * 50 * 0.2125
ymin, ymax = 368 * 50 * 0.2125, 379 * 50 * 0.2125
cell_mask = (
    (adata.obsm["X_spatial"][:, 0] >= xmin)
    & (adata.obsm["X_spatial"][:, 0] <= xmax)
    & (adata.obsm["X_spatial"][:, 1] >= ymin)
    & (adata.obsm["X_spatial"][:, 1] <= ymax)
)

Plotting the connections in the region

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))
sc.pl.embedding(adata, basis="X_spatial", ax=ax, show=False)
fov = Rectangle(
    (xmin, ymin), xmax - xmin, ymax - ymin, edgecolor="red", facecolor="none"
)
ax.add_patch(fov)

In [ ]:
tree = KDTree(adata.obsm["X_spatial"])
distances, indices = tree.query(adata.obsm["X_spatial"][cell_mask], 6)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=((xmax - xmin) / 25, (ymax - ymin) / 25))


ax.scatter(
    adata.obsm["X_spatial"][:, 0],
    adata.obsm["X_spatial"][:, 1],
    s=[80 if i == "T-Cell" else 60 for i in adata.obs["Type"]],
    c=[colors[i] for i in adata.obs["Type"]],
    zorder=10,
)

for neighbor in range(1, indices.shape[1]):
    x_coords = np.array(
        [
            adata.obsm["X_spatial"][indices[:, 0], 0],
            adata.obsm["X_spatial"][indices[:, neighbor], 0],
        ]
    )
    y_coords = np.array(
        [
            adata.obsm["X_spatial"][indices[:, 0], 1],
            adata.obsm["X_spatial"][indices[:, neighbor], 1],
        ]
    )

    ax.plot(
        x_coords[:, adata.obs["Type"][indices[:, 0]] == "T-Cell"],
        y_coords[:, adata.obs["Type"][indices[:, 0]] == "T-Cell"],
        color="firebrick",
        linestyle="-",
        linewidth=1,
        alpha=0.7,
    )
    ax.plot(
        x_coords[:, ~(adata.obs["Type"][indices[:, 0]] == "T-Cell")],
        y_coords[:, ~(adata.obs["Type"][indices[:, 0]] == "T-Cell")],
        color="black",
        linestyle="-",
        linewidth=1,
        alpha=0.2,
    )


ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)
ax.axis("off")
ax.set_aspect("equal", adjustable="box")
fig.tight_layout()